The Scrape Jupyter Notebook scrapes auto feature data and saves it to Consumer_Reports.csv
for future use

# Import Packages

In [ ]:
# Import necessary packages for project
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import time
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import Lasso
from statsmodels.formula.api import ols
from keras import models
from keras import layers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

# Scrape Consumer Reports

Log into Consumer Reports

In [ ]:
# Create Selenium Chrome driver
driver_path = '/Users/blantj/documents/project/car_sales/chromedriver'
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options, executable_path=driver_path)

# Log into Consumer Reports (Input correct username and password)
required_username = None
required_password = None
url = 'https://secure.consumerreports.org/ec/login'
driver.get(url)
login = driver.find_element_by_id(required_username)
login.send_keys('login_required')
password = driver.find_element_by_id(required_password)
password.send_keys('Tigers24')
submit = driver.find_element_by_tag_name("button").click()

Scrape Consumer Reports Review URLs to be scraped

In [ ]:
# From Consumer Reports category pages, scrape review urls and append to review_urls list
ranking_urls = ['https://www.consumerreports.org/cars/types/new/hybrids-evs/ratings',
              'https://www.consumerreports.org/cars/types/new/luxury-cars/ratings',
             'https://www.consumerreports.org/cars/types/new/minivans/ratings',
             'https://www.consumerreports.org/cars/types/new/pickup-trucks/ratings',
             'https://www.consumerreports.org/cars/types/new/sedans/ratings',
             'https://www.consumerreports.org/cars/types/new/small-cars/ratings',
             'https://www.consumerreports.org/cars/types/new/sports-cars/ratings',
             'https://www.consumerreports.org/cars/types/new/suvs/ratings',
             'https://www.consumerreports.org/cars/types/new/wagons/ratings']
review_urls = []
for url in ranking_urls:
    driver.get(url)
    time.sleep(.2)
    review_urls_parent_1 = driver.find_element_by_xpath('/html/body/div[5]/div/div[2]/div/div[4]/table/tbody')
    review_urls_parent_2 = review_urls_parent_1.find_elements_by_tag_name('tr')
    review_urls_parent_3 = []
    for parent_element in review_urls_parent_2:
        review_urls_parent_3.extend(parent_element.find_elements_by_xpath('td[2]/div/div[1]/div[1]/a'))
    for parent_element in review_urls_parent_3:
        review_urls.append(parent_element.get_attribute('href'))
    print(url)

# From Consumer Reports used category pages, scrape review urls and append to used_review_urls list
used_ranking_urls = []
for url in ranking_urls:
    url = url.replace('ratings','')
    used_ranking_urls.append(url.replace('new','used'))
used_review_urls = []
for url in used_ranking_urls:
    driver.get(url)
    time.sleep(.2)
    used_review_urls_parent_1 = driver.find_elements_by_partial_link_text('201')
    for parent_element in used_review_urls_parent_1:
        used_review_urls.append(parent_element.get_attribute('href'))
    print(url)

From scraped review urls, create final list of review urls to scrape

In [ ]:
# Merge new and used review urls to all_review_urls list, while ensuring no duplicates
all_review_urls = list(set(review_urls)) + list(set(used_review_urls))

# Create final_review_urls list to scrape that adds potential missing model years
final_review_urls = []
for url in all_review_urls:
    if int(url[-14:-10]) == 2016:
        final_review_urls.append(url.replace('2016','2015'))
        final_review_urls.append(url.replace('2016','2014'))
        final_review_urls.append(url.replace('2016','2013'))
        final_review_urls.append(url.replace('2016','2012'))
        final_review_urls.append(url.replace('2016','2011'))
        final_review_urls.append(url.replace('2016','2010'))
    elif int(url[-14:-10]) == 2015:
        final_review_urls.append(url.replace('2015','2014'))
        final_review_urls.append(url.replace('2015','2013'))
        final_review_urls.append(url.replace('2015','2012'))
        final_review_urls.append(url.replace('2015','2011'))
        final_review_urls.append(url.replace('2015','2010'))
    elif 2021 > int(url[-14:-10]) > 2009:
        final_review_urls.append(url)
final_review_urls = list(set(final_review_urls))

Create empty df to house scraped vehicle reviews

In [ ]:
#Create empty df with column titles to house scraped vehicle reviews
columns = ('Brand','Model','Year','Overall Score', 'Road Test Score', 'Reliability Score', 'Owner Satisfact Score', 'Warranty',
           'Drivetrain Warranty', 'Corrosion Waranty', 'Roadside Aid', 'Free Maintenance', 'Acceleration Score',
           'Zero To Sixty', 'Transmission Score', 'Fuel Economy Score', 'Overall Fuel Economy', 'City Fuel Economy',
           'Highway Fuel Economy', 'Annual Fuel Cost', 'Braking Score', 'Emergency Handling Score', 'Ride Score',
           'Noise Score', 'Front Seat Score', 'Rear Seat Score', 'Interior Finish Score', 'Trunk Score', 'Seats',
           'Zero To Thirty', 'Fortyfive To Sixty', 'Quarter Mile', 'Quarter Speed', 'Routine Handling Score',
           'Avoidance Max Speed', 'Wet Braking', 'Dry Braking', 'Headlights', 'Driving Position',
           'Front Shoulder Room', 'Front Leg Room', 'Front Head Room', 'Rear Shoulder Room', 'Rear Leg Room',
           'Rear Head Room', 'Third Row Score', 'Third Shoulder Room', 'Third Leg Room', 'Third Head Room',
           'Front Access', 'Rear Access', 'Third Access', 'Usability', 'Climate System', 'Fuel Capacity', 'Fuel Type',
           'Forward Collision Warning', 'City Automatic Braking', 'Highway Automatic Braking', 'Pedestrian Detection',
           'Lane Departure Warning', 'Lane Keep Assist', 'Blind Spot Warning', 'Rear Traffic Warning', 'Rear View Camera',
           'Anti Lock Breaks', 'Traction Control', 'Stability Control', 'Daytime Running Lights', 'Side Airbags',
           'Head Airbags', 'IIHS Moderate Overlap', 'IIHS Driver Overlap', 'IIHS Passenger Overlap', 'IIHS Side Crash',
           'IIHS Rear Crash', 'IIHS Roof Crush', 'Length', 'Width', 'Height', 'Wheelbase', 'Turning Circle', 'Curb Weight',
           'Max Load', 'Towing Capacity', 'Driving Satisfaction', 'Comfort Satisfaction', 'Value Satisfaction',
           'Styling Satisfaction')
df = pd.DataFrame(columns=columns)
df.to_csv("Consumer_Reports.csv")

#Create empty lists for review urls succesfully scraped and not succesfully scraped
scraped_urls = []
unscraped_urls = []

Scrape urls from list of review urls and add to df

In [ ]:
# Check if review url exists and has required data and proceed with scrape if so
count = 0
for review_url in final_review_urls[count:]:
    driver.get(review_url)
    time.sleep(1)
    if (len(driver.find_elements_by_xpath('/html/body/div[3]/div/header/div[4]/div/ul/li[1]/a')) > 0) and (len(driver.find_elements_by_class_name('bar-ratings-chart-member__range')) > 0):
        if driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[3]/div[2]').text!='NA':

#Scrape parts of review url unique to 2020 models            
            if int(review_url[-14:-10]) == 2020:
                overall_score = driver.find_element_by_xpath('/html/body/div[7]/div/div/div/div/div[1]/div[3]/div[2]/div/div/div/div/div/span').text
                road_test_score = driver.find_element_by_class_name('js-bar-chart-mount').text[0:2]
                reliability_score = driver.find_elements_by_class_name('bar-ratings-chart-member__range')[1].text[0]
                owner_satisfact_score = driver.find_elements_by_class_name('bar-ratings-chart-member__range')[2].text[0]
                warranty = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[1]').text[7:]
                drivetrain_warranty = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[2]').text[12:]
                corrosion_waranty = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[3]').text[11:]
                roadside_aid = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[4]').text[14:]
                free_maintenance = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[3]/div/div[2]/ul/li[5]').text[13:]            

#Scrape parts of review url unique to used models            
                else:
                overall_score = None
                road_test_score = None
                reliability_score = driver.find_element_by_xpath('/html/body/div[7]/div/div/div/div/div/div[1]/div[1]/a/div[2]/div/div/div[2]').text[0]
                owner_satisfact_score = driver.find_element_by_xpath('/html/body/div[7]/div/div/div/div/div/div[1]/div[2]/a/div[2]/div').text[0]
                warranty = None
                drivetrain_warranty = None
                corrosion_waranty = None
                roadside_aid = None
                free_maintenance = None 

#Scrape parts of review url common to new and used models                            
            acceleration_score = driver.find_elements_by_class_name('rating-wrapper__data')[0].text[0]
            zero_to_sixty = driver.find_elements_by_class_name('rating-wrapper__data')[1].text[:-4]
            transmission_score = driver.find_elements_by_class_name('display-ratings')[1].text
            fuel_economy_score = driver.find_elements_by_class_name('display-ratings')[2].text
            overal_fuel_economy = driver.find_elements_by_class_name('rating-wrapper__data')[4].text[:-4]
            city_fuel_economy = driver.find_elements_by_class_name('rating-wrapper__data')[5].text[:-4]
            highway_fuel_economy = driver.find_elements_by_class_name('rating-wrapper__data')[6].text[:-4]
            annual_fuel_cost = driver.find_elements_by_class_name('rating-wrapper__data')[7].text[1:]
            braking_score = driver.find_elements_by_class_name('display-ratings')[3].text
            emergency_handling_score = driver.find_elements_by_class_name('display-ratings')[4].text
            ride_score = driver.find_elements_by_class_name('display-ratings')[5].text
            noise_score = driver.find_elements_by_class_name('display-ratings')[6].text
            front_seat_score = driver.find_elements_by_class_name('display-ratings')[7].text
            rear_seat_score = driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[8]/div[2]/div/div/div/div/div[2]').text[0:1]
            interior_finish_score = driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[10]/div[2]/div/div/div/div/div[2]').text[0:1]
            trunk_score = driver.find_element_by_xpath('/html/body/div[8]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[12]/div[2]/div/div/div/div/div[2]').text[0:1]
            seats = driver.find_element_by_xpath('/html/body/div[11]/div/div/div[1]/div/div[2]/div/span').text

#Scrape data from vehicle ratings page associated with review url                            
            driver.get(review_url.replace('overview','ratings'))
            time.sleep(1)
            zero_to_thirty = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-5]
            fortyfive_to_sixty = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2) > div:nth-child(1)').text[:-5]
            quarter_mile = driver.find_element_by_css_selector('div.ratings-sub-row:nth-child(4) > div:nth-child(2) > div:nth-child(1)').text[:-5]
            quarter_speed = driver.find_element_by_css_selector('div.ratings-sub-row:nth-child(5) > div:nth-child(2) > div:nth-child(1)').text[:-4]
            routine_handling_score = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > span:nth-child(1)').text[:1]
            avoidance_max_speed = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(4) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-4]
            wet_braking = data_point = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(5) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text[:-4]
            dry_braking = driver.find_element_by_css_selector('div.ratings-collapse:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(5) > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)').text[:-4]
            headlights = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[2]/div/div/div[7]/div[2]/div/div/div[2]').text[:1]            
            driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/h3/div[2]').click()
            time.sleep(.3)
            driving_position = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[3]/div[2]/div/div/div[2]/span[1]').text[:-1]
            front_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[1]/div[2]/div').text[:-4]
            front_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[2]/div[2]/div').text[:-4]
            front_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[4]/div[3]/div/div[3]/div[2]/div').text[:-4]
            rear_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[1]/div[2]/div').text[:-4]
            rear_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[2]/div[2]/div').text[:-4]
            rear_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[5]/div[3]/div/div[3]/div[2]/div').text[:-4]
            third_row_score = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[2]/div/div/div[2]').text[0:1]
            third_shoulder_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[1]/div[2]/div').text[0:-4]
            third_leg_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[2]/div[2]/div').text[0:-4]
            third_head_room = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[6]/div[3]/div/div[3]/div[2]/div').text[0:-4]
            front_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[7]/div[2]/div/div/div[2]/span[1]').text[0:1]
            rear_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[8]/div[2]/div/div/div[2]').text[0:1]
            third_access = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[9]/div[2]/div/div/div[2]').text[0:1]
            usability = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[10]/div[2]/div/div/div[2]/span[1]').text[0:1]
            climate_system = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[3]/div/div/div[13]/div[2]/div/div/div[2]/span[1]').text[0:1]
            driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/h3/div[2]').click()
            time.sleep(.3)
            fuel_capacity = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/div/div/div[6]/div[2]/div').text[0:-5]
            fuel_type = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[4]/div/div/div[7]/div[2]/div').text
            driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/h3/div[2]').click()
            time.sleep(.3)
            forward_collision_warning = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[1]/div[1]/div[2]/div').text
            city_automatic_emergency_braking = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[1]/div[2]/div[2]/div').text
            highway_automatic_emergency_braking = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[1]/div[3]/div[2]/div').text
            pedestrian_detection = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[1]/div[4]/div[2]/div').text
            lane_departure_warning = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[1]/div[5]/div[2]/div').text
            lane_keeping_assist = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[1]/div[6]/div[2]/div').text
            blind_spot_warning = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[1]/div[7]/div[2]/div').text
            rear_trafic_warning = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[2]/div[1]/div[2]/div').text
            rear_view_camera = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[2]/div[2]/div[2]/div').text
            anti_lock_brakes = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[2]/div[3]/div[2]/div').text
            traction_control = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[2]/div[4]/div[2]/div').text
            stability_control = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[2]/div[5]/div[2]/div').text
            daytime_running_lights = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[1]/div/div[3]/div/div[2]/div[6]/div[2]/div').text
            side_airbags = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[2]/div/div[3]/div/div[1]/div/div[2]/div').text
            head_airbags  = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[2]/div/div[3]/div/div[2]/div/div[2]/div').text
            iihs_moderate_overlap = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[3]/div/div[3]/div/div[1]/div[1]/div[2]/div').text
            iihs_driver_overlap = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[3]/div/div[3]/div/div[1]/div[2]/div[2]/div').text
            iihs_passenger_overlap = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[3]/div/div[3]/div/div[1]/div[3]/div[2]/div').text
            iihs_side_crash = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[3]/div/div[3]/div/div[1]/div[4]/div[2]/div').text
            iihs_rear_crash = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[3]/div/div[3]/div/div[1]/div[5]/div[2]/div').text
            iihs_roof_crush = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[5]/div/div[3]/div/div[3]/div/div[1]/div[6]/div[2]/div').text
            driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/h3/div[2]').click()
            time.sleep(.3)
            length = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/div/div/div[1]/div[2]/div').text[:-4]
            width = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/div/div/div[2]/div[2]/div').text[:-4]
            height = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/div/div/div[3]/div[2]/div').text[:-4]
            wheelbase = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/div/div/div[4]/div[2]/div').text[:-4]
            turning_circle = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/div/div/div[5]/div[2]/div').text[:-4]
            curb_wieght = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/div/div/div[6]/div[2]/div').text[:-4]
            max_load = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/div/div/div[7]/div[2]/div').text[:-4]
            towing_capacity = driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[6]/div/div/div[8]/div[2]/div').text[:-4]            
            driver.get(review_url.replace('overview','owner-satisfaction'))
            time.sleep(1)
            
#Scrape data from vehicle ratings page associated with review url                            
            driving_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[1]/div/div/div[2]/div/div/div/div[2]').text[0:2]
            comfort_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[2]/div/div/div[2]/div/div/div/div[2]').text[0:2]
            value_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[3]/div/div/div[2]/div/div/div/div[2]').text[0:2]
            styling_satisfaction = driver.find_element_by_xpath('/html/body/div[7]/section/article[4]/div/div/div[2]/div/div/div/div[2]').text[0:2]
            model_year = review_url[-14:-10]
            model_brand = review_url.split('/')[4]
            model_name = review_url.split('/')[5]
            new_df = pd.DataFrame([[model_brand,model_name,model_year,overall_score, road_test_score, reliability_score, owner_satisfact_score, warranty,
                         drivetrain_warranty, corrosion_waranty, roadside_aid, free_maintenance, acceleration_score,
                         zero_to_sixty, transmission_score, fuel_economy_score, overal_fuel_economy, city_fuel_economy,
                         highway_fuel_economy, annual_fuel_cost, braking_score, emergency_handling_score, ride_score,
                         noise_score, front_seat_score, rear_seat_score, interior_finish_score, trunk_score, seats,
                         zero_to_thirty, fortyfive_to_sixty, quarter_mile, quarter_speed, routine_handling_score,
                         avoidance_max_speed, wet_braking, dry_braking, headlights, driving_position,
                         front_shoulder_room, front_leg_room, front_head_room, rear_shoulder_room, rear_leg_room,
                         rear_head_room, third_row_score, third_shoulder_room, third_leg_room, third_head_room,
                         front_access, rear_access, third_access, usability, climate_system, fuel_capacity, fuel_type,
                         forward_collision_warning, city_automatic_emergency_braking, highway_automatic_emergency_braking,
                         pedestrian_detection, lane_departure_warning, lane_keeping_assist, blind_spot_warning, 
                         rear_trafic_warning, rear_view_camera, anti_lock_brakes, traction_control, stability_control,
                         daytime_running_lights, side_airbags, head_airbags, iihs_moderate_overlap, iihs_driver_overlap,
                         iihs_passenger_overlap, iihs_side_crash, iihs_rear_crash, iihs_roof_crush, length, width,
                         height, wheelbase, turning_circle, curb_wieght, max_load, towing_capacity, driving_satisfaction,
                         comfort_satisfaction, value_satisfaction, styling_satisfaction]], columns=columns)

#Add scraped review data to existing data df and save to csv                            
            old_df = pd.read_csv("Consumer_Reports.csv", index_col = 0)
            df_save = pd.concat([old_df, new_df])
            df_save.to_csv("Consumer_Reports.csv")
            print(count, review_url)
            scraped_urls.append(review_url)
            count += 1
            
#Track progress through scrape                           
        else:
            print(count, review_url, 'Did not scrape')
            unscraped_urls.append(review_url)
            count += 1
    else:
        print(count, review_url, 'Did not scrape')
        unscraped_urls.append(review_url)
        count += 1     